In [5]:
import re

import requests
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By

from fake_useragent import UserAgent
from fake_useragent import FakeUserAgentError
from math import ceil
from random import randint
from time import sleep
from base64 import b64decode
from io import BytesIO
from PIL import Image
import pandas as pd
from tqdm import tqdm

C:\Users\sante\AppData\Local\Temp\ipykernel_8008\3550857375.py:18: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [6]:
URL = "https://www.avito.ru/all/avtomobili?f=ASgBAgICAUTyCrCKAQ" #не битые авто

In [68]:
def avito_parser(i):
    try:
        ua = UserAgent().chrome
    except FakeUserAgentError:
        ua = "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:84.0) Gecko/20100101 Firefox/84.0"
    options = webdriver.ChromeOptions()
    options.add_argument(f"user-agent={ua}")
    options.add_argument("--disable-blink-features=AutomationControlled")

    driver = webdriver.Chrome(options=options)
    driver.implicitly_wait(10)
    driver.get(URL)

    #count = int(
    #    (
    #        driver.find_element(
    #            by=By.CSS_SELECTOR, value='span[data-marker="page-title/count"]'
    #        ).text
    #    ).replace(" ", "")
    #)

    popular_models = driver.find_element(
            by=By.CSS_SELECTOR, value='div[data-marker="popular-rubricator/links"]'
        ).find_elements(
            by=By.CSS_SELECTOR, value='a[data-marker="popular-rubricator/link"]'
        )
    
    #for i in popular_models:
    #    print(i.text)

    df = pd.DataFrame(columns=['name', 'price', 'params', 'desc', 'region'])

    for _ in range(1):
        model = popular_models[i]
    #for i in tqdm(range(20)):
        #print(model.text)
        url_model = model.get_attribute('href')
        driver.get(url_model)

        for _ in tqdm(range(50)):

            offers = driver.find_elements(
                by=By.CSS_SELECTOR, value='div[data-marker="item"]'
            )

            for offer in offers:
                try:
                    #print(offer.text)
                    avito_id = int(offer.get_attribute("id")[1:])

                    #offer_text = offer.text.split('\n')

                    name = offer.find_element(
                        by=By.CSS_SELECTOR, value='h3[itemprop="name"]'
                    ).text

                    #name = offer_text[0]

                    #print(offer[0])
                    #print(offer[1])
                    #print(offer[2])

                    price = offer.find_element(
                        by=By.CSS_SELECTOR, value='meta[itemprop="price"]'
                    ).get_attribute("content")

                    #price = offer_text[1]

                    params = offer.find_element(
                        by=By.CSS_SELECTOR, value='p[data-marker="item-specific-params"]'
                    ).text

                    #params = re.search(r'([^,\n]+,){4}[^,\n]+', offer.text, re.IGNORECASE)[0]

                    desc = offer.find_element(
                        by=By.CSS_SELECTOR, value='div[class="iva-item-descriptionStep-C0ty1"]'
                    ).text
                    #desc = offer_text[-3]

                    region = offer.find_element(
                        by=By.CSS_SELECTOR, value='div[class="geo-root-zPwRk"]'
                    ).text
                    #region = offer_text[-2]

                    #print(f"name: {name}")
                    #print(f"price: {price}")
                    #print(f"params: {params}")
                    #print(f"desc: {desc}")
                    #print(f"region: {region}")
                    df.loc[avito_id, 'name':'region'] = [name, price, params, desc, region]

                except Exception as ex:
                    print(ex)

            try:    
                driver.find_element(
                    by=By.CSS_SELECTOR, value='a[data-marker="pagination-button/nextPage"]'
                ).click()

                print(f'len_df: {len(df)}')

                rand_sleep = randint(25, 49)
                sleep(rand_sleep / 10)

            except Exception as ex:
                    print(ex)

        df.to_csv(f'./datasets/data{i}.csv')


    driver.quit()

    return df

In [15]:
test = 'Renault Kaptur 1.6 CVT, 2017, 145 586 км\n900 000 ₽\nНаписать\nПоказать телефон\n145 586 км, 1.6 CVT (114 л.с.), внедорожник, передний, бензин\nКаптюры в наличии, есть из чего выбрать! Все авто на новой шипованной резине. Где посмотреть. Санкт-Петербург, Шушары, Московское шоссе, 15, а. Вт–сб с 10:00 до 19:00. Про машину. Цена с включённым Ндс 20%. Автомобиль в наличии с Птс, выдача в день обращения. Оплата Сбп, по карте без комиссии или безналичным расчётом. Если у вас ИП или ООО, поможем с лизингом. Полная исправность и прозрачная история обслуживания по регламенту производителя в течение всего пробега. Никаких дополнительных платежей, стоимость автомобиля как в объявлении. Подойдёт для курьерских служб, корпоративного парка, автошкол, таксопарков и как личный автомобиль.\nАвтодилер'

params = re.search(r'([^,\n]+,){4}[^,\n]+', test, re.IGNORECASE)
params[0]

'145 586 км, 1.6 CVT (114 л.с.), внедорожник, передний, бензин'

In [71]:
i = [22, 23, 24, 25, 26, 27]
for k in i:
    print(k)
    a = avito_parser(k)

22


  0%|          | 0/50 [00:00<?, ?it/s]

Message: no such element: Unable to locate element: {"method":"css selector","selector":"div[class="iva-item-descriptionStep-C0ty1"]"}
  (Session info: chrome=121.0.6167.140); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF6C1845E42+3538674]
	(No symbol) [0x00007FF6C1464C02]
	(No symbol) [0x00007FF6C1315AEB]
	(No symbol) [0x00007FF6C135BF4E]
	(No symbol) [0x00007FF6C135C0CC]
	(No symbol) [0x00007FF6C13512EC]
	(No symbol) [0x00007FF6C137F0EF]
	(No symbol) [0x00007FF6C135124A]
	(No symbol) [0x00007FF6C137F2C0]
	(No symbol) [0x00007FF6C139BDE3]
	(No symbol) [0x00007FF6C137EE53]
	(No symbol) [0x00007FF6C134F514]
	(No symbol) [0x00007FF6C1350631]
	GetHandleVerifier [0x00007FF6C1876CAD+3738973]
	GetHandleVerifier [0x00007FF6C18CC506+4089270]
	GetHandleVerifier [0x00007FF6C18C4823+4057299]
	GetHandleVerifier [0x00007FF6C1595C49+720121]
	(No symbol) [0x0000

  2%|▏         | 1/50 [00:41<34:14, 41.92s/it]

len_df: 99


...

 96%|█████████▌| 48/50 [18:31<02:10, 65.48s/it]

Message: no such element: Unable to locate element: {"method":"css selector","selector":"div[class="iva-item-descriptionStep-C0ty1"]"}
  (Session info: chrome=121.0.6167.140); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF6C1845E42+3538674]
	(No symbol) [0x00007FF6C1464C02]
	(No symbol) [0x00007FF6C1315AEB]
	(No symbol) [0x00007FF6C135BF4E]
	(No symbol) [0x00007FF6C135C0CC]
	(No symbol) [0x00007FF6C13512EC]
	(No symbol) [0x00007FF6C137F0EF]
	(No symbol) [0x00007FF6C135124A]
	(No symbol) [0x00007FF6C137F2C0]
	(No symbol) [0x00007FF6C139BDE3]
	(No symbol) [0x00007FF6C137EE53]
	(No symbol) [0x00007FF6C134F514]
	(No symbol) [0x00007FF6C1350631]
	GetHandleVerifier [0x00007FF6C1876CAD+3738973]
	GetHandleVerifier [0x00007FF6C18CC506+4089270]
	GetHandleVerifier [0x00007FF6C18C4823+4057299]
	GetHandleVerifier [0x00007FF6C1595C49+720121]
	(No symbol) [0x0000

 98%|█████████▊| 49/50 [19:15<00:58, 58.77s/it]

Message: no such element: Unable to locate element: {"method":"css selector","selector":"div[class="iva-item-descriptionStep-C0ty1"]"}
  (Session info: chrome=121.0.6167.140); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF6C1845E42+3538674]
	(No symbol) [0x00007FF6C1464C02]
	(No symbol) [0x00007FF6C1315AEB]
	(No symbol) [0x00007FF6C135BF4E]
	(No symbol) [0x00007FF6C135C0CC]
	(No symbol) [0x00007FF6C13512EC]
	(No symbol) [0x00007FF6C137F0EF]
	(No symbol) [0x00007FF6C135124A]
	(No symbol) [0x00007FF6C137F2C0]
	(No symbol) [0x00007FF6C139BDE3]
	(No symbol) [0x00007FF6C137EE53]
	(No symbol) [0x00007FF6C134F514]
	(No symbol) [0x00007FF6C1350631]
	GetHandleVerifier [0x00007FF6C1876CAD+3738973]
	GetHandleVerifier [0x00007FF6C18CC506+4089270]
	GetHandleVerifier [0x00007FF6C18C4823+4057299]
	GetHandleVerifier [0x00007FF6C1595C49+720121]
	(No symbol) [0x0000

100%|██████████| 50/50 [20:11<00:00, 24.22s/it]


In [45]:
df = pd.DataFrame(columns=['name', 'price', 'params', 'desc', 'region'])
a = 'audi'
df.to_csv(f'./datasets/data_{a}.csv')

In [27]:

year = re.search(r'Год выпуска: (\d+)\n', a).group(1)
year

'2023'

In [35]:
mileage = re.search(r'Пробег: (\d+ ?\d*) км\n', a).group(1).replace(' ', '')
mileage

'5'

In [32]:
users = re.search(r'Владельцев по ПТС: (\d+)\n', a).group(1)
users

'1'

In [37]:
condition = re.search(r'Состояние: ([\w ]+)\n', a, re.IGNORECASE).group(1)
condition

'Не битый'

In [38]:
engine_type = re.search(r'Тип двигателя: (\w+)\n', a, re.IGNORECASE).group(1)
engine_type

'Бензин'

In [40]:
engine_capacity = re.search(r'Объём двигателя: ([\d.]+) л\n', a, re.IGNORECASE).group(1)
engine_capacity

'3'

In [41]:
transmission = re.search(r'Коробка передач: (\w+)\n', a, re.IGNORECASE).group(1)
transmission

'Автомат'

In [43]:
equipment = re.search(r'Комплектация: (.+)\n', a, re.IGNORECASE).group(1)
equipment

'M Sport'

In [44]:
body_type = re.search(r'Тип кузова: (.+)\n', a, re.IGNORECASE).group(1)
body_type

'Внедорожник 5-дверный'

In [46]:
wheel = re.search(r'Руль: (\w+)\n', a, re.IGNORECASE).group(1)
wheel

'Левый'

In [47]:
modification = re.search(r'Модификация: (.+)\n', a, re.IGNORECASE).group(1)
modification

'40i 3.0 xDrive Steptronic (381 л.с.)'

In [18]:
params_elems = a.find_element(
    by=By.CSS_SELECTOR, value='div[data-marker="item-view/item-params"]'
).text

params_elems


KeyboardInterrupt: 